<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/java_dsm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [7]:
!pip install plyj

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
  Created wheel for plyj: filename=plyj-0.1-py3-none-any.whl size=17422 sha256=198d27d407baa5aa1bf4f469b4d265af0b595215d68431e62cf45d4f56638516
  Stored in directory: /root/.cache/pip/wheels/54/9a/2b/e5280bbfa303fea2aa489ea4e5c1a09dd6f41230257cd0cafa
Successfully built plyj


In [66]:
import os
from git import Repo
from plyj.parser import Parser
import numpy as np

In [65]:
#!rm -rf promocity

In [67]:
# Clone the repository
repo_url = "https://github.com/armandossrecife/promocity.git"
repo_folder = "promocity"
repo = Repo.clone_from(repo_url, repo_folder)

In [68]:
# Initialize a dictionary to store dependencies
dependency_matrix = {}

In [69]:
# Initialize the Java parser
parser = Parser()

In [70]:
java_files = []
for root, dirs, files in os.walk(repo_folder):
    for file in files:
        if file.endswith(".java"):
            java_files.append(os.path.join(root, file))

In [71]:
for each in java_files:
  print(each)

promocity/src/main/java/ufc/cmu/promocity/backend/BackendApplication.java
promocity/src/main/java/ufc/cmu/promocity/backend/JerseyConfiguration.java
promocity/src/main/java/ufc/cmu/promocity/backend/GenericExceptionMapper.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/MyTracking.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Book.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Users.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Track.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Coupon.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/AbstractModel.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Promotion.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/MyStores.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Category.java
promocity/src/main/java/ufc/cmu/promocity/backend/model/Store.java
promocity/src/main/java/ufc/cmu/promocity/backend/repository/CouponsRepository.j

In [72]:
# Iterate through Java files
for java_file in java_files:
    with open(os.path.join('/content', java_file), "r") as file:
        content = file.read()
        try:
          tree = parser.parse_string(content)
          imports = [imp.name.value for imp in tree.import_declarations]

          # Iterate through all type declarations (classes and interfaces)
          for type_declaration in tree.type_declarations:
            if hasattr(type_declaration, 'name') and type_declaration.name:
              type_name = type_declaration.name
              dependency_matrix[type_name] = imports
        except Exception as e:
          print(f"Error parsing {java_file}: {e}")

In [45]:
print(dependency_matrix)

{'BackendApplication': ['org.springframework.boot.SpringApplication', 'org.springframework.boot.autoconfigure.SpringBootApplication', 'ufc.cmu.promocity.backend.context'], 'JerseyConfiguration': ['javax.annotation.PostConstruct', 'javax.ws.rs.ApplicationPath', 'org.glassfish.jersey.server.ResourceConfig', 'org.springframework.context.annotation.Configuration', 'ufc.cmu.promocity.backend.context.PromotionArea', 'ufc.cmu.promocity.backend.controller.BookController', 'ufc.cmu.promocity.backend.controller.CouponController', 'ufc.cmu.promocity.backend.controller.PrincipalController', 'ufc.cmu.promocity.backend.controller.PromotionController', 'ufc.cmu.promocity.backend.controller.StoreController', 'ufc.cmu.promocity.backend.controller.UserController', 'ufc.cmu.promocity.backend.report.ReportApplication'], 'GenericExceptionMapper': ['javax.ws.rs.core.Response', 'javax.ws.rs.ext.ExceptionMapper', 'javax.ws.rs.ext.Provider'], 'MyTracking': ['java.util.LinkedList', 'java.util.List', 'javax.pers

In [73]:
# Create a list of all unique class and interface names
all_types = set(dependency_matrix.keys())
for dependencies in dependency_matrix.values():
    all_types.update(dependencies)

In [44]:
print(all_types)

{'ufc.cmu.promocity.backend.repository.PromotionsRepository', 'ufc.cmu.promocity.backend.report.ReportCoupon', 'java.util.List', 'javax.persistence.OneToMany', 'AbstractModel', 'UserController', 'MyStoresRepository', 'org.springframework.jdbc.support.CustomSQLExceptionTranslatorRegistrar', 'org.hibernate.annotations.Type', 'java.util.Calendar', 'org.hibernate.annotations.LazyCollectionOption', 'LocationStore', 'ufc.cmu.promocity.backend.model', 'ufc.cmu.promocity.backend.model.MyStores', 'ufc.cmu.promocity.backend.model.Coupon', 'ufc.cmu.promocity.backend.service.BookService', 'org.springframework.security.config.annotation.web.configuration.EnableWebSecurity', 'GeographicArea', 'ufc.cmu.promocity.backend.model.Track', 'ufc.cmu.promocity.backend.repository.MyStoresRepository', 'BookController', 'PromotionArea', 'org.springframework.boot.SpringApplication', 'UserNotFoundException', 'java.util.Set', 'java.net.URI', 'ufc.cmu.promocity.backend.report.ReportUser', 'Store', 'MyTrackingReposi

In [74]:
# Create an empty dependency matrix
num_types = len(all_types)
matrix = np.zeros((num_types, num_types), dtype=int)

In [76]:
# Fill in the dependency matrix
type_index_mapping = {type_name: index for index, type_name in enumerate(all_types)}
lista_types = list(type_index_mapping.keys())
for type_name, imports in dependency_matrix.items():
    type_index = type_index_mapping[type_name]
    for dependency in imports:
        dependency_index = type_index_mapping.get(dependency, -1)
        if dependency_index != -1:
            matrix[type_index, dependency_index] = 1

In [81]:
with open('mydsm.txt', mode='w') as f:
  for i in range(0,len(matrix)):
    for j in range(0,len(matrix)):
      if matrix[i, j] == 1:
        conteudo = f'{lista_types[i]}, {lista_types[j]}, {matrix[i,j]}'
        conteudo = conteudo + '\n'
        f.write(conteudo)